In [ ]:
import os
from openai import OpenAI
from pathlib import Path
from datetime import datetime
from collections import defaultdict
from dotenv import load_dotenv
load_dotenv()

In [ ]:
cwd = Path(os.getcwd())
data_path = cwd.parent / "data" / "pliki_z_fabryki" / "do-not-share"
assert data_path.exists()

## Embedding with LlamaIndex

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex

Settings.embed_model = OpenAIEmbedding()

documents = SimpleDirectoryReader(data_path).load_data()
index = VectorStoreIndex.from_documents(documents=documents, embed_model=Settings.embed_model)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("co to jest EIZ")

In [ ]:
for src in response.source_nodes:
    print(src)

## Embedding manually

In [ ]:
ai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
# process documents one by one
# some metadata can be seeded programmatically without calling llm
# some will be generated by llm as preprocessing

rich_documents = defaultdict(dict)

for file in data_path.iterdir():
    with open(file, "r") as f:
        text = f.read()
        rich_documents[file.name]["metadata"] = {
                "file_name": file.name,
                "date": datetime.strptime(file.name.split(".")[0], "%Y_%m_%d").date().strftime("%Y-%m-%d"),
                "length": len(text),
            }
        rich_documents[file.name]["content"] = text

def extract_main_topic_with_llm(text):
    sys_prompt = f"""
    <instrukcje>
    W tekście przesłanym przez użytkownika jest zawarta nazwa urządzania militarnego.
    Wskaż jaka to nazwa i zwróć ją bez żadnych dodatkowych informacji i komentarzy.
    Jest to zawsze jedna nazwa.
    Nazwa w tekście może być w innym przypadku niż mianownik - w takim przypadku zwróć przekształconą nazwę w mianowniku.
    Nazwa zawsze znajduje się blisko początku dokumentu.
    </instrukcje>
    <przykłady>
    Tekst: Przeprowadzono testy bulbulatora jonowego. Wyniki są obiecujące.
    Odpowiedź: bulbulator jonowy
    Tekst: W wyniku testów stwierdzono, że kryptonizer jest gotowy do produkcji seryjnej.
    Odpowiedź: kryptonizer
    Tekst: Wyrzutnia ABC-100
    Odpowiedź: wyrzutnia ABC-100
    </przykłady>
    """
    return ai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": text}
        ]
    ).choices[0].message.content


In [45]:
# add autogen uuids

import uuid

for key, document in rich_documents.items():
    document["metadata"]["uuid"] = str(uuid.uuid4())

In [ ]:
# use llm to enrich documents with main topic

for key, document in rich_documents.items():
    document["metadata"]["main_topic"] = extract_main_topic_with_llm(document["content"])


In [46]:
# embed the docs into qdrant

from qdrant_client import QdrantClient

qdrant = QdrantClient(":memory:")

docs = [
    document["content"]
    for _, document in rich_documents.items()
]

meta = [
    {
        "date": document["metadata"]["date"],
        "length": document["metadata"]["length"],
        "main_topic": document["metadata"]["main_topic"]
    }
    for _, document in rich_documents.items()
]

ids = [
    document["metadata"]["uuid"]
    for _, document in rich_documents.items()
]

qdrant.add(
    collection_name="s03e02",
    documents=docs,
    metadata=meta,
    ids=ids
)

['4bcbd1ea-b076-46a4-a7ec-e8e22fa13e5e',
 'ffcf1a43-5c4d-45a3-beae-94e24c62ad74',
 'cfdbe167-bd91-46d4-b5de-5aa8c6408cdf',
 'faeb6d36-bdd9-4bf8-ab5f-acbf11631483',
 '7977a860-e152-4098-a42d-414c16664e76',
 '8130b659-ffd3-4be6-a723-d02cc8b14f5e',
 'b9b8ed25-c0f6-443e-9e25-88cf5a7ddeac',
 'c57462a1-d8dd-4ad8-9dbf-254747e16bf9',
 '5d877bda-2007-47ea-8a20-9908f76cfa1d',
 '154edca7-7c86-4c34-b400-ec021b9fe02e',
 'db3eaec0-4ac1-4c60-a951-8f8baeb79644',
 '17ff9abc-48c6-4bfc-8967-7198ec59ed0f',
 'b09712bd-0cc4-430c-aefd-dec044b273b0',
 '4a2cdafa-0ae9-45ba-a48b-f4cf5175e4c9',
 '419c117b-ef02-4859-a249-8c76faf1daab',
 '567f9004-0aaf-4e51-be24-a6ad3a82b563',
 '9243a8e3-38ce-4db3-ab6d-e6d4ac900179',
 '00521028-9440-44aa-a446-bae85cbb49b4',
 '250e6aff-d263-4fc7-9545-2cc3fcc2b9e1',
 '75ea418b-6bcf-41f8-aef3-37e535ffef40',
 'f108706c-4e66-491d-9117-14c841905079',
 'aaa463a5-509d-4c92-9f5c-9b2aec07e545',
 '3bcaa167-b220-4126-abf1-7db947d4fccf']

In [69]:
search_result = qdrant.query(
    collection_name="s03e02",
    query_text="w raporcie z którego dnia znajduje się wzmianka o kradzieży prototypu broni",
    limit=20
)

In [ ]:
for res in search_result:
    print(f"Date: {res.metadata['date']}, score: {res.score}")

# the result we need is not top ranked

Date: 2024-03-31, score: 0.8989213246894683
Date: 2024-01-29, score: 0.8889468643176821
Date: 2024-02-21, score: 0.8807557942665944
Date: 2024-02-01, score: 0.8790207616365366
Date: 2024-01-27, score: 0.8774150221774103
Date: 2024-03-19, score: 0.8759423238160611
Date: 2024-04-27, score: 0.8701192074513983
Date: 2024-02-15, score: 0.8700239253311656
Date: 2024-05-08, score: 0.8681198140472859
Date: 2024-01-17, score: 0.864943854040213
Date: 2024-03-12, score: 0.8643186968911939
Date: 2024-06-02, score: 0.863538909239175
Date: 2024-03-02, score: 0.8629254288840875
Date: 2024-03-18, score: 0.8611756013408354
Date: 2024-01-08, score: 0.860655542984372
Date: 2024-05-31, score: 0.860359549409335
Date: 2024-02-11, score: 0.8601826504168983
Date: 2024-04-18, score: 0.859383416777385
Date: 2024-07-05, score: 0.8579143196532768
Date: 2024-05-14, score: 0.8564758056480242


## Wzbogacanie metadanych

W pierwotnej wersji nie ma wystarczająco dobrych metadanych aby search zadziałał. Szukany rezultat jest na 3-5 miejscu.

Możliwe rozwiązania:
- Dodać słowa kluczowe w ramach techniki contextual retrieval. Tworząc embeddingi tym razem kompresujemy całość tj. tekst + słowa kluczowe.
- Wprowadzić re-rank.
- Wykorzystać inny embedding -> https://huggingface.co/spaces/mteb/leaderboard

In [63]:
def extract_keywords_with_llm(text):
    sys_prompt = f"""
    <instrukcje>
    W tekście przesłanym przez użytkownika jest zawarta nazwa urządzania militarnego.
    Wyselekcjonuj z tekstu słowa kluczowe.
    Zwracaj uwagę przede wszystkim na nazwy własne, nazwy urządzeń, nazwy miejsc, nazwy jednostek organizacyjnych, zdarzenia, czynności.
    Zamień każde słowo kluczowe na formę podstawową.
    Zwróć słowa kluczowe w postaci listy oddzielonej przecinkami. Nie stosuj żadnego innego formatowania.
    </instrukcje>
    """

    return ai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": text}
        ]
    ).choices[0].message.content

# use llm to enrich documents with keywords

for key, document in rich_documents.items():
    document["metadata"]["keywords"] = extract_keywords_with_llm(document["content"])

In [82]:
# embed the docs into qdrant

import json
from qdrant_client import QdrantClient

qdrant_rich = QdrantClient(":memory:")

def enrich_content(content: str, meta: dict):
    """Adds metadata to the content."""
    return f"""
    <metadata>
    {json.dumps(meta)}
    </metadata>
    <content>
    {content}
    </content>
    """

docs = [
    enrich_content(document["content"], document["metadata"])
    for _, document in rich_documents.items()
]

meta = [
    {
        "date": document["metadata"]["date"],
        "length": document["metadata"]["length"],
        "main_topic": document["metadata"]["main_topic"],
        "keywords": document["metadata"]["keywords"]
    }
    for _, document in rich_documents.items()
]

ids = [
    document["metadata"]["uuid"]
    for _, document in rich_documents.items()
]

ids = qdrant_rich.add(
    collection_name="s03e02_enriched",
    documents=docs,
    metadata=meta,
    ids=ids
)

In [87]:
search_result = qdrant_rich.query(
    collection_name="s03e02_enriched",
    query_text="w raporcie z którego dnia znajduje się wzmianka o kradzieży prototypu broni",
    limit=5
)

In [88]:
for res in search_result:
    print(f"Date: {res.metadata['date']}, score: {res.score}, keywords: {res.metadata['keywords']}")

Date: 2024-01-17, score: 0.8731580272940108, keywords: testy, broń, Irradiator XR-25, promieniowanie, materiały, ekosystemy, wnioski, modyfikacja, system, chłodzenie, eksploatacja, prototyp, wersja, rozpylacz, jony, efekty, otoczenie
Date: 2024-04-27, score: 0.8719703375670659, keywords: Wysokotemperaturowy Pojemnik Kinetyczny, raport, testy, konstrukcja, generowanie, ekstremalne temperatury, wydobywanie, energia kinetyczna, temperatury, stopnie Celsjusza, stopienie, materiały, twardość, stal, wyniki, niszczenie, wrogie pojazdy, przekształcanie, teren, nieprzyjazny, przeciwnik, wnioski, potrzeba, rozwijanie, trwałość, efektywność, warunki atmosferyczne, roboty badawcze, modyfikacje, portfel zastosowań, wojskowe operacje, postapokaliptyczne
Date: 2024-01-29, score: 0.8712197477551457, keywords: broń, kryptonim, plazmowy, korpus, zniszczenia, testy, wyniki, zasięg, działanie, moc, wystrzał, temperatura, materiał, wnioski, użycie, przestrzeń, nadzór, katastrofa, ekologiczny, powód, badani

## Dalsze usprawnienia

Wzbogacenie metadanych nie zadziałało. Nie pomoże nawet filtrowanie po słowach kluczowych - bo w top 5 rezultatów nie ma dobrego kandydata.

Sprawdzamy inny embedding - jina-embeddings-v3.

In [89]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

class JinaEmbeddings:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("jinaai/jina-embeddings-v3")
        self.model = AutoModel.from_pretrained("jinaai/jina-embeddings-v3")

    def encode(self, texts):
        if isinstance(texts, str):
            texts = [texts]

        encoded = self.tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            model_output = self.model(**encoded)
        
        embeddings = model_output.last_hidden_state[:, 0, :]

        embeddings = embeddings.numpy()
        embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

        return embeddings
    
embeddings = JinaEmbeddings()

A new version of the following files was downloaded from https://huggingface.co/jinaai/xlm-roberta-flash-implementation:
- configuration_xlm_roberta.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/jinaai/xlm-roberta-flash-implementation:
- rotary.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/jinaai/xlm-roberta-flash-implementation:
- xlm_padding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/jinaai/xlm-roberta-flash-implementation:
- mlp.

In [96]:
qdrant_jina = QdrantClient(":memory:")

In [97]:
from qdrant_client.http import models

qdrant_jina.create_collection(
    collection_name="s03e02_enriched_jina",
    vectors_config=models.VectorParams(size=1024, distance=models.Distance.COSINE)
)

True

In [108]:
def prepare_content():
    def enrich_content(content: str, meta: dict):
        """Adds metadata to the content."""
        return f"""
        <metadata>
        {json.dumps(meta)}
        </metadata>
        <content>
        {content}
        </content>
        """

    docs = [
        enrich_content(document["content"], document["metadata"])
        for _, document in rich_documents.items()
    ]

    meta = [
        {
            "date": document["metadata"]["date"],
            "length": document["metadata"]["length"],
            "main_topic": document["metadata"]["main_topic"],
            "keywords": document["metadata"]["keywords"]
        }
        for _, document in rich_documents.items()
    ]

    ids = [
        document["metadata"]["uuid"]
        for _, document in rich_documents.items()
    ]

    return docs, meta, ids

In [112]:
docs, meta, ids = prepare_content()

In [125]:
qdrant_jina.upsert(
    collection_name="s03e02_enriched_jina",
    points=[
        models.PointStruct(
            id=id,
            vector=embeddings.encode(doc).tolist(),
            payload={"doc": doc, "date": meta["date"], "main_topic": meta["main_topic"]}
        )
        for doc, id, meta in (zip(docs, ids, meta))
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [126]:
query = "W raporcie, z którego dnia znajduje się wzmianka o kradzieży prototypu broni?"

results = qdrant_jina.search(
    collection_name="s03e02_enriched_jina",
    query_vector=embeddings.encode(query)[0],
    limit=1
)

In [129]:
answer = results[0].payload["date"]
print(answer)

2024-02-21


In [130]:
from aidevs3.poligon import send

load_dotenv()

key = os.environ.get("AG3NTS_API_KEY")
url = f"{os.environ.get("AG3NTS_CENTRALA_URL")}/report"


res = send(url, answer=answer, apikey=key, task="wektory")
print(res)

{'code': 0, 'message': '{{FLG:ZLODZIEJ}}'}
